## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [32]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self._codigo = self._validar_codigo(codigo)
        self._nombre = self._validar_nombre(nombre)
        self._precio = self._validar_precio_inicial(precio)
        self._tipo = self._validar_tipo(tipo)
        self._cantidad = self._validar_cantidad_inicial(cantidad)

    def _validar_codigo(self, codigo):
        return codigo if codigo.isdigit() and len(codigo) == 4 else '0000'

    def _validar_nombre(self, nombre):
        return nombre if 1 <= len(nombre) <= 100 else 'Sin nombre'

    def _validar_precio_inicial(self, precio):
        return precio if 10 <= precio <= 10000 else 10

    def _validar_tipo(self, tipo):
        return tipo[:20] if 0 <= len(tipo) <= 20 else ''

    def _validar_cantidad_inicial(self, cantidad):
        return cantidad if 0 <= cantidad <= 100 else 0

    def valorTotal(self):
        return self._precio * self._cantidad

    @property
    def codigo(self):
        return self._codigo

    @property
    def nombre(self):
        return self._nombre

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, nuevo_precio):
        if 10 <= nuevo_precio <= 10000:
            self._precio = nuevo_precio

    @property
    def tipo(self):
        return self._tipo

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, nueva_cantidad):
        if 0 <= nueva_cantidad <= 100:
            self._cantidad = nueva_cantidad

In [57]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

AttributeError: 'Producto' object has no attribute 'valorTotal'

# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [34]:
from abc import ABC, abstractmethod

class Oferta(ABC):
    def __init__(self, descripcion="Oferta sin descripción", codigos=None, tipos=None):
        self.descripcion = descripcion
        self.codigos = codigos or []
        self.tipos = tipos or []

    def esAplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    @abstractmethod
    def calcularDescuento(self, producto, cantidad):
        pass

class OfertaDescuento(Oferta):
    def __init__(self, descuento, **kwargs):
        super().__init__(**kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            return int((producto.precio * cantidad) * (self.descuento / 100))
        return 0

class Oferta2x1(Oferta):
    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            pares_completos = cantidad // 2
            return pares_completos * producto.precio
        return 0


class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad

In [35]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [52]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def calcularDescuento(self, producto, cantidad):
        mejor_descuento = 0
        for oferta in self.ofertas:
            descuento_actual = oferta.calcularDescuento(producto, cantidad)
            if descuento_actual > mejor_descuento:
                mejor_descuento = descuento_actual
        return mejor_descuento

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    def guardar(self, archivo):
        with open(archivo, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, mode='r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Saltar la cabecera
            for row in reader:
                codigo, nombre, precio, tipo, cantidad = row
                producto = Producto(codigo, nombre, int(precio), tipo, int(cantidad))
                catalogo.agregar(producto)
        return catalogo

    def vender(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad

    def informe(self):
        tipos = set(producto.tipo for producto in self.productos)
        informe = (
            f"Cantidad de productos: {self.cantidadProductos}\n"
            f"Cantidad de unidades: {self.cantidadUnidades}\n"
            f"Precio Promedio: {self.valorTotal / self.cantidadUnidades:.2f}\n"
            f"Valor total: {self.valorTotal:.2f}\n"
            f"Tipos de productos: {', '.join(tipos)}\n"
            "Ofertas:\n"
        )
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                informe += "Oferta 2x1\n"
            elif isinstance(oferta, OfertaDescuento):
                informe += f"Oferta {oferta.descuento}% de descuento\n"
        return informe

In [53]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [42]:
class Cliente:
    def __init__(self, nombre, cuit):
        self.__nombre = ''
        self.__cuit = ''
        self.establecer_nombre(nombre)
        self.establecer_cuit(cuit)

    def obtener_nombre(self):
        return self.__nombre

    def establecer_nombre(self, valor):
        if valor and 1 <= len(valor) <= 100:
            self.__nombre = valor

    def obtener_cuit(self):
        return self.__cuit

    def establecer_cuit(self, valor):
        if self.__validar_cuit(valor):
            self.__cuit = valor

    def __validar_cuit(self, cuit):
        partes = cuit.split('-')
        return (len(partes) == 3 and
                len(partes[0]) == 2 and
                len(partes[1]) == 8 and
                len(partes[2]) == 1 and
                all(parte.isdigit() for parte in partes))

    nombre = property(obtener_nombre, establecer_nombre)
    cuit = property(obtener_cuit, establecer_cuit)

In [43]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [54]:
class Factura:
    ultimaFactura_num = 0  

    @classmethod
    def ultimaFactura(cls, numero):
        cls.ultimaFactura_num = numero

    @classmethod
    def nuevoNumero(cls):
        cls.ultimaFactura_num += 1
        return cls.ultimaFactura_num

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = Factura.nuevoNumero()
        self.productos = []
        self.cantidades = []
        self.descuentos_aplicados = [] 

    def agregar(self, producto, cantidad):
        if producto.codigo in [p.codigo for p in self.productos]:
            indice = [p.codigo for p in self.productos].index(producto.codigo)
            self.cantidades[indice] += cantidad
        else:
            self.productos.append(producto)
            self.cantidades.append(cantidad)
        self.catalogo.vender(producto, cantidad)

    # El resto de los métodos permanecen sin cambios
    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(self.cantidades)

    @property
    def subtotal(self):
        return sum(producto.precio * cantidad for producto, cantidad in zip(self.productos, self.cantidades))

    @property
    def descuentos(self):
        total_descuento = 0
        self.descuentos_aplicados = []  
        for producto, cantidad in zip(self.productos, self.cantidades):
            ofertas_aplicables = [oferta for oferta in self.catalogo.ofertas if oferta.esAplicable(producto)]
            descuentos = [oferta.calcularDescuento(producto, cantidad) for oferta in ofertas_aplicables]
            if descuentos:
                max_descuento = max(descuentos)
                total_descuento += max_descuento
                for oferta in ofertas_aplicables:
                    if oferta.calcularDescuento(producto, cantidad) == max_descuento:
                        self.descuentos_aplicados.append(self._descripcion_oferta(oferta, producto))
                        break
        return total_descuento

    def _descripcion_oferta(self, oferta, producto):
        if isinstance(oferta, OfertaDescuento):
            return f"Descuento {oferta.descuento}% en {producto.nombre}"
        elif isinstance(oferta, Oferta2x1):
            return f"Oferta 2x1 en {producto.nombre}"
        return "Oferta especial"

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        salida = f"Cliente: {self.cliente.nombre}\n"
        for producto, cantidad in zip(self.productos, self.cantidades):
            salida += f"{cantidad} {producto.nombre}\n"
        salida += f"SUBTOTAL: {self.subtotal:.2f}\n"
        for descripcion in self.descuentos_aplicados:
            salida += f"{descripcion}\n"
        salida += f"DESCUENTO: {self.descuentos:.2f}\n"
        salida += f"TOTAL: {self.total:.2f}\n"
        return salida

In [55]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [56]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
